In [91]:
from shapely.geometry import Polygon, LineString, Point
import pandas as pd
import numpy as np
import re
import geopandas as gpd
import matplotlib.pyplot as plt

In [92]:
# Everything must be in epsg = 32617 or 3857
# 3857 makes it a smaller area than 32617
# 32617 is closest to LANDAREA (when it exists)

In [93]:
station_data = pd.read_excel("../station-movers-V1.xlsx")
da2021 = pd.read_csv("../../data_raw/cleaned_data/2021_census_data.csv")
da2016 = pd.read_csv("../../data_raw/cleaned_data/2016_census_data.csv")
da2011 = pd.read_csv("../../data_raw/cleaned_data/2011_census_data.csv")
da2006 = pd.read_csv("../../data_raw/cleaned_data/2006_census_data.csv")
da2001 = pd.read_csv("../../data_raw/cleaned_data/2001_census_data.csv")
ct1996 = pd.read_csv("../../data_raw/cleaned_data/1996_census_data.csv")
ct1991 = pd.read_csv("../../data_raw/cleaned_data/1991_census_data.csv")
ct1986 = pd.read_csv("../../data_raw/cleaned_data/1986_census_data.csv")
ct1981 = pd.read_csv("../../data_raw/cleaned_data/1981_census_data.csv")
ct1976 = pd.read_csv("../../data_raw/cleaned_data/1976_census_data.csv")
boundaries_2021 = gpd.read_file("../../data_raw/boundary_data/lda_000b21a_e/lda_000b21a_e.shp")
boundaries_2016 = gpd.read_file("../../data_raw/boundary_data/lda_000b16a_e/lda_000b16a_e.shp")
boundaries_2011 = gpd.read_file("../../data_raw/boundary_data/gda_000b11a_e/gda_000b11a_e.shp")
boundaries_2006 = gpd.read_file("../../data_raw/boundary_data/gda_000b06a_e/gda_000b06a_e.shp")
boundaries_2001 = gpd.read_file("../../data_raw/boundary_data/gda_000b02m_e/gda_000b02m_e.MID")
boundaries_1996 = gpd.read_file("../../data_raw/CT_data/1996/ct_1996.geojson")
boundaries_1991 = gpd.read_file("../../data_raw/CT_data/1991/ct_1991.geojson")
boundaries_1986 = gpd.read_file("../../data_raw/CT_data/1986/ct_1986.geojson")
boundaries_1981 = gpd.read_file("../../data_raw/CT_data/1981/ct_1981.geojson")
boundaries_1976 = gpd.read_file("../../data_raw/CT_data/1976/ct_1976.geojson")

/opt/anaconda3/envs/SofC/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: ../../data_raw/boundary_data/gda_000b06a_e/gda_000b06a_e.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [94]:
station_gdf = gpd.GeoDataFrame(station_data, geometry=gpd.points_from_xy(station_data['X'], station_data['Y']))
station_gdf.set_crs(epsg=4326, inplace=True)
station_gdf = station_gdf.to_crs(epsg=32617)
station_gdf["buffer"] = station_gdf.buffer(800)

In [95]:
boundaries_2021["DAUID"] = boundaries_2021["DAUID"].astype(int)
boundaries_2021 = boundaries_2021.to_crs(epsg=32617)
boundaries_2021 = boundaries_2021[["DAUID","geometry"]]

In [96]:
boundaries_2016["DAUID"] = boundaries_2016["DAUID"].astype(int)
boundaries_2016 = boundaries_2016.to_crs(epsg=32617)
boundaries_2016 = boundaries_2016[["DAUID","geometry"]]

In [97]:
boundaries_2011["DAUID"] = boundaries_2011["DAUID"].astype(int)
boundaries_2011 = boundaries_2011.to_crs(epsg=32617)
boundaries_2011 = boundaries_2011[["DAUID","geometry"]]

In [98]:
boundaries_2006["DAUID"] = boundaries_2006["DAUID"].astype(int)
boundaries_2006 = boundaries_2006.to_crs(epsg=32617)
boundaries_2006 = boundaries_2006[["DAUID","geometry"]]

In [99]:
boundaries_2001["DAUID"] = boundaries_2001["DAUID"].astype(int)
boundaries_2001 = boundaries_2001.to_crs(epsg=32617)
boundaries_2001 = boundaries_2001[["DAUID","geometry"]]

In [100]:
boundaries_1996 = boundaries_1996.to_crs(epsg=32617)
boundaries_1996["geosid"] = boundaries_1996["geosid"].astype(float)
boundaries_1996["area"] = boundaries_1996["areakm"] * 1000000 

In [101]:
boundaries_1991 = boundaries_1991.to_crs(epsg=32617)
boundaries_1991["geosid"] = boundaries_1991["geosid"].astype(float)
boundaries_1991["area"] = boundaries_1991["areakm"] * 1000000 

In [102]:
boundaries_1986 = boundaries_1986.to_crs(epsg=32617)
boundaries_1986["geosid"] = boundaries_1986["geosid"].astype(float)
boundaries_1986["area"] = boundaries_1986["areakm"] * 1000000 

In [103]:
boundaries_1981 = boundaries_1981.to_crs(epsg=32617)
boundaries_1981["geosid"] = boundaries_1981["geosid"].astype(float)
boundaries_1981["area"] = boundaries_1981["areakm"] * 1000000 

In [104]:
boundaries_1976 = boundaries_1976.drop(index=[2310, 2484])

boundaries_1976 = boundaries_1976.to_crs(epsg=32617)
boundaries_1976["geosid"] = boundaries_1976["geosid"].astype(float)
boundaries_1976["area"] = boundaries_1976["areakm"] * 1000000 

In [105]:
mapping_years = {
    2021: [boundaries_2021, da2021],
    2016: [boundaries_2016, da2016],
    2011: [boundaries_2011, da2011],
    2006: [boundaries_2006, da2006],
    2001: [boundaries_2001, da2001],
    
    1996: [boundaries_1996, ct1996],
    1991: [boundaries_1991, ct1991],
    1986: [boundaries_1986, ct1986],
    1981: [boundaries_1981, ct1981],
    1976: [boundaries_1976, ct1976],
}

## Functions

In [106]:
def ten_yr_span(dates, my_list):
    top_year = max(my_list)
    bottom_year = min(my_list)

    closest = min(my_list, key=lambda x: abs(x - dates))
    if closest == bottom_year:
        return closest, closest + 5, closest + 10
    elif closest == top_year:
        return closest - 10, closest - 5, closest
    return closest - 5, closest, closest + 5
        

In [107]:
def get_dates(data, date_col):
    """
    Get the 10_yr_span of dates
    Add this is before/after date columns to the dataframe
    Saves it in_place
    """
    dates = pd.to_datetime(data[date_col])
    years = dates.dt.year
    list_of_years = [1976, 1981, 1986, 1991, 1996, 2001, 2006, 2011, 2016, 2021]
    set_of_years = years.apply(lambda x: ten_yr_span(x, list_of_years))
    years_df = pd.DataFrame(set_of_years.tolist(), columns=[f"before_{date_col}", f"middle_{date_col}", f"after_{date_col}"])
    return_df = pd.concat([data, years_df], axis=1)
    return return_df

In [108]:
def find_overlap(buffer_area, boundary_data):
    """
    Given a buffer area, find the overlapping regions
    Get the census data of the relevant year
    1) Find the overlapping regions
    2) Calculate the area of overlap
    3) Find the proportion of each areas (if proportion > 1; proportion = 1)
    """
    overlapping = boundary_data[boundary_data.intersects(buffer_area)].reset_index()    
    overlapping.loc[:,"overlap"] = overlapping.geometry.intersection(buffer_area).area
    overlapping["proportion"] = overlapping["overlap"] / overlapping.area
    overlapping["proportion"] = overlapping["proportion"].where(overlapping["proportion"] <= 1, 1)
    return overlapping

In [109]:
def find_data(boundary_data, census_data, year):
    """
    1) Merge the boundary and census data
    2) Mulitply the census data by the proportion
    3) Sum all the census data 
    4) Return a row of values
    """
    if year >= 2001:
        combined = boundary_data.merge(census_data, left_on = "DAUID", right_on = "GeoUID")
        useful = combined.iloc[:,4:]
        useful_portioned = useful.mul(useful["proportion"], axis=0)
        summed_row = useful_portioned.sum()[3:]
        return summed_row
    else:
        combined = boundary_data.merge(census_data, on = "geosid")
        useful = combined.iloc[:,16:]
        useful_portioned = useful.mul(useful["proportion"], axis=0)
        summed_row = useful_portioned.sum()[1:]
        return summed_row

In [110]:
def join_rows(array_of_series):
    df = pd.DataFrame(array_of_series)
    return df

In [111]:
station_gdf = get_dates(station_gdf, "opening_date")

/var/folders/qd/w0jg42cx76sc6s81d9zfb7t80000gn/T/ipykernel_61134/808158239.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates = pd.to_datetime(data[date_col])


In [112]:
my_row = station_gdf.iloc[27,:]

In [113]:
before_year = my_row["before_opening_date"]
middle_year = my_row["middle_opening_date"]
after_year = my_row["after_opening_date"]
area = my_row["buffer"]

In [118]:
after_b_data = mapping_years[after_year][0]
after_c_data = mapping_years[after_year][1]

In [119]:
overlapping = after_b_data[after_b_data.intersects(area)].reset_index()    
overlapping.loc[:,"overlap"] = overlapping.geometry.intersection(area).area
overlapping["proportion"] = overlapping["overlap"] / overlapping.area
overlapping["proportion"] = overlapping["proportion"].where(overlapping["proportion"] <= 1, 1)

In [131]:
overlapping

,index,DAUID,geometry,overlap,proportion
0,23061,35200265,"POLYGON ((634176.951 4848908.895, 634157.098 4...",281.921984,0.001918
1,23063,35200267,"POLYGON ((633701.951 4848909.898, 633702.304 4...",73276.799153,0.514502
2,23064,35200268,"POLYGON ((634026.418 4848288.172, 633771.578 4...",2445.418303,0.054093
3,23066,35200270,"POLYGON ((635122.365 4847827.317, 635134.106 4...",127120.666265,0.098703
4,23067,35200271,"POLYGON ((633519.27 4847954.342, 633503.63 484...",85257.750305,1.000000
5,23068,35200272,"POLYGON ((633519.27 4847954.342, 633523.364 48...",147329.505040,1.000000
6,23069,35200273,"POLYGON ((633321.954 4848225.901, 633289.398 4...",45612.628205,1.000000
7,23070,35200275,"POLYGON ((633081.235 4847610.392, 633029.953 4...",145824.123652,0.990908
8,23071,35200278,"POLYGON ((632697.95 4847728.899, 632727.948 48...",60382.217132,0.411237
9,23072,35200279,"POLYGON ((633078.849 4848044.666, 633032.205 4...",154519.023493,1.000000


In [132]:
combined

,index,DAUID,geometry,overlap,proportion,GeoUID,CMA_UID,Population Density per square kilometre,Dwellings,Total Occupied Private Dwellings,...,Semi-detached house,Row house,"Apartment, duplex","Apartment, building that has five or more storeys","Apartment, building that has fewer than five storeys",Other single-attached house,Movable dwelling,Average number of bedrooms per dwelling,Owned,Rented
0,23061,35200265,"POLYGON ((634176.951 4848908.895, 634157.098 4...",281.921984,0.001918,35200265,35535.0,3517.006803,187,175.0,...,40.0,0.0,0.0,0.0,55.0,0.0,0.0,2.9,135.0,40.0
1,23063,35200267,"POLYGON ((633701.951 4848909.898, 633702.304 4...",73276.799153,0.514502,35200267,35535.0,8170.141082,514,505.0,...,40.0,0.0,10.0,425.0,10.0,0.0,0.0,2.3,430.0,80.0
2,23064,35200268,"POLYGON ((634026.418 4848288.172, 633771.578 4...",2445.418303,0.054093,35200268,35535.0,11853.162318,261,250.0,...,0.0,40.0,0.0,205.0,0.0,0.0,0.0,2.3,225.0,25.0
3,23066,35200270,"POLYGON ((635122.365 4847827.317, 635134.106 4...",127120.666265,0.098703,35200270,35535.0,1046.275167,432,415.0,...,0.0,130.0,0.0,285.0,0.0,0.0,0.0,2.3,130.0,285.0
4,23067,35200271,"POLYGON ((633519.27 4847954.342, 633503.63 484...",85257.750305,1.000000,35200271,35535.0,12475.085004,568,415.0,...,0.0,40.0,0.0,380.0,0.0,0.0,0.0,1.7,40.0,370.0
5,23068,35200272,"POLYGON ((633519.27 4847954.342, 633523.364 48...",147329.505040,1.000000,35200272,35535.0,13339.666169,648,630.0,...,0.0,0.0,0.0,630.0,0.0,0.0,0.0,1.7,15.0,610.0
6,23069,35200273,"POLYGON ((633321.954 4848225.901, 633289.398 4...",45612.628205,1.000000,35200273,35535.0,30089.853167,505,485.0,...,0.0,0.0,0.0,490.0,0.0,0.0,0.0,1.6,0.0,485.0
7,23070,35200275,"POLYGON ((633081.235 4847610.392, 633029.953 4...",145824.123652,0.990908,35200275,35535.0,3043.064801,158,155.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,3.7,150.0,10.0
8,23071,35200278,"POLYGON ((632697.95 4847728.899, 632727.948 48...",60382.217132,0.411237,35200278,35535.0,2662.037037,136,135.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.7,135.0,0.0
9,23072,35200279,"POLYGON ((633078.849 4848044.666, 633032.205 4...",154519.023493,1.000000,35200279,35535.0,3713.527851,219,220.0,...,0.0,110.0,0.0,0.0,0.0,0.0,0.0,3.4,200.0,15.0


In [129]:
combined = overlapping.merge(after_c_data, left_on = "DAUID", right_on = "GeoUID")
useful = combined.iloc[:,4:]
useful_portioned = useful.mul(useful["proportion"], axis=0)
summed_row = useful_portioned.sum()[3:]

In [130]:
summed_row

Population Density per square kilometre                 183918.146639
Dwellings                                                 6154.813305
Total Occupied Private Dwellings                          5846.112389
Single-detached house                                      563.002724
Semi-detached house                                         20.656807
Row house                                                  729.977453
Apartment, duplex                                           54.845826
Apartment, building that has five or more storeys         4495.145494
Apartment, building that has fewer than five storeys         5.250533
Other single-attached house                                  5.822144
Movable dwelling                                             0.000000
Average number of bedrooms per dwelling                     32.122935
Owned                                                     1796.180484
Rented                                                    4049.560319
dtype: float64

## Running Everything

In [21]:
station_gdf = get_dates(station_gdf, "opening_date")

/var/folders/qd/w0jg42cx76sc6s81d9zfb7t80000gn/T/ipykernel_38805/808158239.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates = pd.to_datetime(data[date_col])


In [22]:
each_row = []

for i, row in station_gdf.iterrows():
    before_year = row["before_opening_date"]
    middle_year = row["middle_opening_date"]
    after_year = row["after_opening_date"]
    area = row["buffer"]

    before_b_data = mapping_years[before_year][0]
    before_c_data = mapping_years[before_year][1]
    middle_b_data = mapping_years[middle_year][0]
    middle_c_data = mapping_years[middle_year][1]
    after_b_data = mapping_years[after_year][0]
    after_c_data = mapping_years[after_year][1]
  
    before_overlap = find_overlap(area, before_b_data)
    before_row = find_data(before_overlap, before_c_data, before_year)
    middle_overlap = find_overlap(area, middle_b_data)
    middle_row = find_data(middle_overlap, middle_c_data, middle_year)
    after_overlap = find_overlap(area, after_b_data)
    after_row = find_data(after_overlap, after_c_data, after_year)

    before_row.index = ["Before " + str(i) for i in before_row.index]
    middle_row.index = ["Middle " + str(i) for i in middle_row.index]
    after_row.index = ["After " + str(i) for i in after_row.index]

    full_row = pd.concat([before_row, middle_row, after_row])
    each_row.append(full_row)

In [23]:
full = join_rows(each_row)

In [24]:
ordered_columns = [
        'Before Population Density per square kilometre', 'Before Dwellings',
        'Before Total Occupied Private Dwellings',
        'Before Single-detached house', 'Before Semi-detached house',
        #'Before Other attached dwelling',
        'Before Row house', 'Before Apartment',
        'Before Apartment, duplex',
        'Before Apartment, building that has fewer than five storeys',
        'Before Apartment, building that has five or more storeys',
        'Before Other dwelling', 'Before Other single-attached house',
        'Before Movable dwelling', 
        'Before No bedrooms',
        'Before 0 to 1 bedroom', 
        'Before 1 bedroom', 'Before 2 bedrooms',
        'Before 3 bedrooms', 
        'Before 4 bedrooms', 
        'Before 4 or more bedrooms', 
        'Before 5 or more bedrooms', 
        'Before Average number of bedrooms per dwelling', 
        'Before Owned',
        'Before Rented', 

        'Middle Population Density per square kilometre', 'Middle Dwellings',
        'Middle Total Occupied Private Dwellings',
        'Middle Single-detached house', 'Middle Semi-detached house',
        'Middle Other attached dwelling',
        'Middle Row house', #'Middle Apartment',
        'Middle Apartment, duplex',
        'Middle Apartment, building that has fewer than five storeys',
        'Middle Apartment, building that has five or more storeys',
        'Middle Other dwelling', 'Middle Other single-attached house',
        'Middle Movable dwelling', 
        'Middle No bedrooms',
        'Middle 0 to 1 bedroom', 
        'Middle 1 bedroom', 'Middle 2 bedrooms',
        'Middle 3 bedrooms', 
        'Middle 4 bedrooms', 
        'Middle 4 or more bedrooms', 
        'Middle 5 or more bedrooms',
        'Middle Average number of bedrooms per dwelling', 
        'Middle Owned',
        'Middle Rented',
        
        'After Population Density per square kilometre', 'After Dwellings',
        'After Total Occupied Private Dwellings',
        'After Single-detached house', 'After Semi-detached house',
        'After Other attached dwelling',
        'After Row house', 
        'After Apartment, duplex',
        'After Apartment, building that has fewer than five storeys',
        'After Apartment, building that has five or more storeys',
        'After Other dwelling', 'After Other single-attached house',
        'After Movable dwelling', 
        'After No bedrooms',
        'After 0 to 1 bedroom', 
        'After 1 bedroom', 'After 2 bedrooms',
        'After 3 bedrooms',
        # 'After 4 bedrooms', 
        'After 4 or more bedrooms', 
        # 'After 5 or more bedrooms',
        'After Average number of bedrooms per dwelling',
        'After Owned',
        'After Rented'
]

full = full[ordered_columns]

In [25]:
combined_df = pd.concat([station_gdf, full], axis=1)

In [26]:
combined_df.to_csv('../data/tod-on-main.csv', index=False)